In [7]:
# Imports 
import pandas as pd
import itertools 
import networkx as nx
# import matplotlib.pyplot as plt
# import pytrends
# import musicbrainzngs as m
import numpy as np

# from pytrends.request import TrendReq
from os import path
from PIL import Image
# from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [8]:
artist_genres_df = pd.read_json("../data/artist_genres.json")

In [9]:
artist_genres_df.head()

,artist,genres
0,"""Groove"" Holmes","[hard bop, jazz and blues, soul jazz]"
1,"""Weird Al"" Yankovic","[accordion, american, comedy, comedy rock, par..."
2,'N Sync & Gloria Estefan,"[adult contemporary, american, bolero, conga, ..."
3,'N Sync Featuring Nelly,"[boy band, dance-pop, pop, pop and chart, pop ..."
4,'Til Tuesday,"[alternative rock, new wave]"


In [156]:
import ciso8601
import time
import re
def separate_artists(df):
    split_string = ' and |, | AND | & | featuring | FEATURING | x | X | feat | ft |,| with | feauring | feat. | / '
    ret_list = []
    res_check = set()
    for index, colab in df.iterrows():
        date = ciso8601.parse_datetime(colab['date'])
        res = [time.mktime(date.timetuple())]
        artists = re.split(split_string, colab['artist'], flags=re.IGNORECASE)
        if tuple(artists) not in res_check:
            res_check.add(tuple(artists))
            res.append(artists)
            ret_list.append(res)
    return ret_list

In [178]:
df = pd.read_csv("../data/billboard.csv", index_col = 0)
df = df[df.date >= "2000-01-01"]

separated_artists = separate_artists(df)
collabs = [y for [x,y] in separated_artists if len(y)>1]

In [179]:
unique_artists = list(np.unique([artist for l in collabs for artist in l]))

In [180]:
unique_artists;
artists_by_id = dict({a:b for a,b in zip(unique_artists,list(range(0,len(unique_artists))))})
artists_by_id = {key:[value,0.0] for (key,value) in artists_by_id.items()}

In [181]:
artists_by_id

{' J. Cole': [0, 0.0],
 ' Nicki Minaj': [1, 0.0],
 '$NOT': [2, 0.0],
 '& Cardi B Or Nas': [3, 0.0],
 '& Jack Harlow': [4, 0.0],
 '& Lil Wayne': [5, 0.0],
 "'N Sync": [6, 0.0],
 '112': [7, 0.0],
 '2 Chainz': [8, 0.0],
 '2 Pistols': [9, 0.0],
 '21 Savage': [10, 0.0],
 '24kGoldn': [11, 0.0],
 '2Cellos': [12, 0.0],
 '2Pac': [13, 0.0],
 '3LW': [14, 0.0],
 '3OH!3': [15, 0.0],
 '42 Dugg': [16, 0.0],
 '5 Seconds Of Summer': [17, 0.0],
 '50 Cent': [18, 0.0],
 '6LACK': [19, 0.0],
 '6ix9ine': [20, 0.0],
 '7 Aurelius': [21, 0.0],
 '98 Degrees': [22, 0.0],
 '99 Percent': [23, 0.0],
 '9lokknine': [24, 0.0],
 'A Boogie Wit da Hoodie': [25, 0.0],
 'A Day To Remember': [26, 0.0],
 'A Great Big World': [27, 0.0],
 'A R Rahman': [28, 0.0],
 'A$AP Ferg': [29, 0.0],
 'A$AP Rocky': [30, 0.0],
 'A-Trak + Milo': [31, 0.0],
 'A.B. Quintanilla': [32, 0.0],
 'A7S': [33, 0.0],
 'AJ': [34, 0.0],
 'ATR Son Son': [35, 0.0],
 'AURORA': [36, 0.0],
 'Aaliyah': [37, 0.0],
 'Aaron Lewis': [38, 0.0],
 'Aaron Lewis Of Stai

In [182]:
for l in collabs:
    for artist in l:
        artists_by_id[artist][1] +=1

In [183]:
top100=sorted(artists_by_id.items(), key=lambda x: x[1][1], reverse=True)[:100];
top100=dict(top100);
top100

{'Lil Wayne': [957, 109.0],
 'Drake': [459, 108.0],
 'Nicki Minaj': [1186, 81.0],
 'Chris Brown': [315, 71.0],
 'Kanye West': [831, 67.0],
 'Future': [570, 57.0],
 'Young Thug': [1761, 57.0],
 'Lil Baby': [936, 53.0],
 'Rick Ross': [1333, 48.0],
 'Justin Bieber': [811, 47.0],
 'Travis Scott': [1646, 47.0],
 'Eminem': [495, 44.0],
 'Big Sean': [176, 41.0],
 'DJ Khaled': [379, 40.0],
 'Ludacris': [993, 40.0],
 'Meek Mill': [1074, 40.0],
 'T-Pain': [1503, 40.0],
 '2 Chainz': [8, 36.0],
 'Lil Uzi Vert': [956, 33.0],
 'T.I.': [1506, 33.0],
 '21 Savage': [10, 32.0],
 'DaBaby': [390, 31.0],
 'Juice WRLD': [803, 31.0],
 'Pitbull': [1258, 31.0],
 'Akon': [51, 30.0],
 'Lil Durk': [941, 30.0],
 'Rihanna': [1336, 30.0],
 'The Weeknd': [1596, 30.0],
 'Ty Dolla $ign': [1665, 30.0],
 'Cardi B': [265, 29.0],
 'Tyga': [1666, 29.0],
 'Ariana Grande': [99, 28.0],
 'Wiz Khalifa': [1713, 28.0],
 'Gunna': [618, 27.0],
 'Kendrick Lamar': [858, 27.0],
 'Snoop Dogg': [1446, 27.0],
 'Usher': [1679, 27.0],
 'Pol

In [184]:
artist_genres_df = pd.read_json("../data/artist_genres.json")

In [185]:
def determine_genre(subgenres) :
    for subgenre in subgenres :
        for genre, subgrs in aggregate_genres.items():
            if subgenre in subgrs:
                return genre
    return np.NaN

In [186]:
# Source : https://github.com/kevinschaich/billboard 
aggregate_genres = {
    "rock": ["symphonic rock", "jazz-rock", "heartland rock", "rap rock", "garage rock", 
           "folk-rock", "roots rock", "adult alternative pop rock", "rock roll", "punk rock", 
           "arena rock", "pop-rock", "glam rock", "southern rock", "indie rock", "funk rock", 
           "country rock", "piano rock", "art rock", "rockabilly", "acoustic rock", "progressive rock", 
           "folk rock", "psychedelic rock", "rock & roll", "blues rock", "alternative rock", 
           "rock and roll", "soft rock", "rock and indie", "hard rock", "pop/rock", "pop rock", 
           "rock", "classic pop and rock", "psychedelic", "british psychedelia", "punk", "metal", "heavy metal", 
             "brutal death metal"],
"alternative/indie": ["adult alternative pop rock", "alternative rock", "alternative metal", 
                       "alternative", "lo-fi indie", "indie", "indie folk", "indietronica", 
                       "indie pop", "indie rock", "rock and indie"],
"electronic/dance": ["dance and electronica", "electro house", "electronic", "electropop", 
                      "progressive house", "hip house", "house", "eurodance", "dancehall", "dance", "trap", "funk carioca"],
"soul": ["psychedelic soul", "deep soul", "neo-soul", "neo soul", "southern soul", 
          "smooth soul", "blue-eyed soul", "soul and reggae", "soul", "funk"],
"classical/soundtrack": ["classical", "orchestral", "film soundtrack", "composer"],
"pop": ["country-pop", "latin pop", "classical pop", "pop-metal", "orchestral pop", 
         "instrumental pop", "indie pop", "sophisti-pop", "pop punk", "pop reggae", "noah cyrus"
         "britpop", "traditional pop", "power pop", "sunshine pop", "baroque pop", "alternative r&b",
         "synthpop", "art pop", "teen pop", "psychedelic pop", "folk pop", "country pop", 
         "pop rap", "pop soul", "pop and chart", "dance-pop", "pop", "top 40", "american idol", 'breakeven', 'm2m'],
"hip-hop/rnb": ["conscious hip hop", "east coast hip hop", "hardcore hip hop", 
                 "west coast hip hop", "hiphop", "southern hip hop", "hip-hop", 
                 "hip hop", "hip hop rnb and dance hall", "contemporary r b", 
                 "gangsta rap", "rapper", "rap", "rhythm and blues", "contemporary rnb", 
                 "contemporary r&b", "rnb", "rhythm & blues","r&b", "blues", "mumble rapper", 'michael jackson'],
"disco": ["disco"],
"swing":  ["swing"],
"folk": ["contemporary folk", "folk", "bachata", "latin", "romanian"],
"country": ["country rock", "country-pop", "country pop", "contemporary country", "country"],
"jazz": ["vocal jazz", "jazz", "jazz-rock"],
"religious": ["christian", "christmas music", "gospel"],
"blues": ["delta blues", "rock blues", "urban blues", "electric blues",
           "acoustic blues", "soul blues", "country blues", "jump blues", 
           "classic rock", "blues rock", "jazz and blues", "piano blues", 
           "british blues", "british rhythm & blues", "rhythm and blues", "blues", "blues rock", "rhythm & blues"],
"reggae": ["reggae fusion", "roots reggae", "reggaeton", "pop reggae", "reggae", "soul and reggae"]}

In [187]:
artist_genres_df['genre'] = artist_genres_df['genres'].apply(lambda x: determine_genre(x))

In [188]:
top100

{'Lil Wayne': [957, 109.0],
 'Drake': [459, 108.0],
 'Nicki Minaj': [1186, 81.0],
 'Chris Brown': [315, 71.0],
 'Kanye West': [831, 67.0],
 'Future': [570, 57.0],
 'Young Thug': [1761, 57.0],
 'Lil Baby': [936, 53.0],
 'Rick Ross': [1333, 48.0],
 'Justin Bieber': [811, 47.0],
 'Travis Scott': [1646, 47.0],
 'Eminem': [495, 44.0],
 'Big Sean': [176, 41.0],
 'DJ Khaled': [379, 40.0],
 'Ludacris': [993, 40.0],
 'Meek Mill': [1074, 40.0],
 'T-Pain': [1503, 40.0],
 '2 Chainz': [8, 36.0],
 'Lil Uzi Vert': [956, 33.0],
 'T.I.': [1506, 33.0],
 '21 Savage': [10, 32.0],
 'DaBaby': [390, 31.0],
 'Juice WRLD': [803, 31.0],
 'Pitbull': [1258, 31.0],
 'Akon': [51, 30.0],
 'Lil Durk': [941, 30.0],
 'Rihanna': [1336, 30.0],
 'The Weeknd': [1596, 30.0],
 'Ty Dolla $ign': [1665, 30.0],
 'Cardi B': [265, 29.0],
 'Tyga': [1666, 29.0],
 'Ariana Grande': [99, 28.0],
 'Wiz Khalifa': [1713, 28.0],
 'Gunna': [618, 27.0],
 'Kendrick Lamar': [858, 27.0],
 'Snoop Dogg': [1446, 27.0],
 'Usher': [1679, 27.0],
 'Pol

In [168]:
artist_genres_df[artist_genres_df['artist'] == 'DJ Khaled']

,artist,genres,genre


In [189]:
for key, value in top100.items():
    try:
        value.append(artist_genres_df[artist_genres_df['artist'] == key].genre.values[0])
    except IndexError:
        print(key)

DJ Khaled
Ty Dolla $ign
Polo G
Jay-Z
David Guetta
Lil Jon
Metro Boomin
Timbaland
P. Diddy
Nate Dogg
Adam Levine
Bow Wow
The Creator
Tyler
Zedd


In [139]:
aggregate_genres.keys()

dict_keys(['rock', 'alternative/indie', 'electronic/dance', 'soul', 'classical/soundtrack', 'pop', 'hip-hop/rnb', 'disco', 'swing', 'folk', 'country', 'jazz', 'religious', 'blues', 'reggae'])

In [190]:
top100['DJ Khaled'].append('hip-hop/rnb')
top100['Ty Dolla $ign'].append('hip-hop/rnb')
top100['Polo G'].append('hip-hop/rnb')
top100['Jay-Z'].append('hip-hop/rnb')
top100['David Guetta'].append('electronic/dance')
top100['Lil Jon'].append('hip-hop/rnb')
top100['Metro Boomin'].append('hip-hop/rnb')
top100['Timbaland'].append('hip-hop/rnb')
top100['P. Diddy'].append('hip-hop/rnb')
top100['Nate Dogg'].append('hip-hop/rnb')
top100['Adam Levine'].append('pop')
top100['Bow Wow'].append('hip-hop/rnb')
top100['The Creator'].append('hip-hop/rnb')
top100['Tyler'].append('hip-hop/rnb')
top100['Zedd'].append('electronic/dance')

In [191]:
top100

{'Lil Wayne': [957, 109.0, 'hip-hop/rnb'],
 'Drake': [459, 108.0, 'pop'],
 'Nicki Minaj': [1186, 81.0, 'electronic/dance'],
 'Chris Brown': [315, 71.0, 'hip-hop/rnb'],
 'Kanye West': [831, 67.0, 'electronic/dance'],
 'Future': [570, 57.0, 'hip-hop/rnb'],
 'Young Thug': [1761, 57.0, 'hip-hop/rnb'],
 'Lil Baby': [936, 53.0, 'hip-hop/rnb'],
 'Rick Ross': [1333, 48.0, 'hip-hop/rnb'],
 'Justin Bieber': [811, 47.0, 'religious'],
 'Travis Scott': [1646, 47.0, 'hip-hop/rnb'],
 'Eminem': [495, 44.0, 'hip-hop/rnb'],
 'Big Sean': [176, 41.0, 'hip-hop/rnb'],
 'DJ Khaled': [379, 40.0, 'hip-hop/rnb'],
 'Ludacris': [993, 40.0, 'hip-hop/rnb'],
 'Meek Mill': [1074, 40.0, 'hip-hop/rnb'],
 'T-Pain': [1503, 40.0, 'hip-hop/rnb'],
 '2 Chainz': [8, 36.0, 'hip-hop/rnb'],
 'Lil Uzi Vert': [956, 33.0, 'hip-hop/rnb'],
 'T.I.': [1506, 33.0, 'hip-hop/rnb'],
 '21 Savage': [10, 32.0, 'hip-hop/rnb'],
 'DaBaby': [390, 31.0, 'hip-hop/rnb'],
 'Juice WRLD': [803, 31.0, 'hip-hop/rnb'],
 'Pitbull': [1258, 31.0, 'electronic

In [201]:
import os 

f = open("nodes.json", "w")

f.write("[")


for artist, elems in top100.items():
    f.write('{"id": ' + str(elems[0]) )
    f.write(', "label": "' + artist  )
    f.write('", "group": "' + elems[2] + '"},\n')

f.seek(f.tell() - 1, os.SEEK_SET)
f.write('')
f.seek(f.tell() - 1, os.SEEK_SET)
f.write('')
f.seek(f.tell() - 1, os.SEEK_SET)
f.write('')

f.write("]")
f.close()

In [203]:
f = open("edges.json", "w")

f.write("[")

for pair in list(itertools.combinations(list(top100.keys()), 2)):
    for collab in collabs:
        if pair[0] in collab and pair[1] in collab:
                f.write('{"from": ' + str(artists_by_id[pair[0]][0]) +', "to": ' + str(artists_by_id[pair[1]][0]) + '},\n')
                break
        
f.seek(f.tell() - 1, os.SEEK_SET)
f.write('')
f.seek(f.tell() - 1, os.SEEK_SET)
f.write('')
f.seek(f.tell() - 1, os.SEEK_SET)
f.write('')

f.write("]")
f.close()

In [ ]:
most_collab_artists = [a for (a,b) in artists_by_id.items()]

In [ ]:
most_collab_artists

In [ ]:
list(itertools.combinations(collabs[6], 2))[0][0]

In [ ]:
artists_by_id['J. Cole']

In [ ]:
included_artists = most_collab_artists.copy()
included_artists = set(included_artists)

for art in most_collab_artists:
    for collab in collabs: 
        if art in collab:
            included_artists = included_artists.union(set(collab))
included_artists = list(included_artists)

In [ ]:
artists_by_id = dict({a:b for a,b in zip(included_artists,list(range(0,len(included_artists))))})
artists_by_id 

In [ ]:
f = open("nodes.txt", "w")

f.write("var nodes = new vis.DataSet([\n")

for artist, i in top100.items():
    f.write("\t{id: "+str(i[0])+", label: '"+artist+"'},\n") 
        
f.write("]);")
f.close()

In [ ]:
'Lil Wayne' in ['Lil Wayne', 'asdsd'] and 'Li' in ['Lil Wayne', 'asdsd']

In [ ]:
 for pair in list(itertools.combinations(list(top100.keys()), 2)):
        print(pair)

In [ ]:
artists_by_id['Lil Wayne'][0]

In [ ]:
f = open("edges.txt", "w")


f.write("var edges = new vis.DataSet([\n")

for pair in list(itertools.combinations(list(top100.keys()), 2)):
    for collab in collabs:
        if pair[0] in collab and pair[1] in collab:
                f.write("\t{from: "+str(artists_by_id[pair[0]][0])+', to: '+str(artists_by_id[pair[1]][0])+"},\n")
                break
        
f.write("]);")
f.close()

In [ ]:
collabs[6]

In [ ]:
f = open("hey.txt", "w")

f.write("series: [")

for genre in aggregate_genres.keys():
    f.write("\t{\n")
    f.write("\tname : "+"'"+genre+"'"+",\n")
    f.write("\tdata: [")
    
    for subgenre in aggregate_genres[genre] : 
        f.write("{\n")
        f.write("\t\tname: "+"'"+subgenre+"',\n")
        f.write("\t\tvalue: "+str(subgenres_popularity[subgenre])+"\n")
        f.write("\t},")
    
    f.write('\t]\n')   #  closing data 
    f.write("\t},\n")

f.write("]")

f.close()

In [ ]:
# Source : https://github.com/kevinschaich/billboard 
aggregate_genres = {
    "rock": ["symphonic rock", "jazz-rock", "heartland rock", "rap rock", "garage rock", 
           "folk-rock", "roots rock", "adult alternative pop rock", "rock roll", "punk rock", 
           "arena rock", "pop-rock", "glam rock", "southern rock", "indie rock", "funk rock", 
           "country rock", "piano rock", "art rock", "rockabilly", "acoustic rock", "progressive rock", 
           "folk rock", "psychedelic rock", "rock & roll", "blues rock", "alternative rock", 
           "rock and roll", "soft rock", "rock and indie", "hard rock", "pop/rock", "pop rock", 
           "rock", "classic pop and rock", "psychedelic", "british psychedelia", "punk", "metal", "heavy metal", 
             "brutal death metal"],
"alternative/indie": ["adult alternative pop rock", "alternative rock", "alternative metal", 
                       "alternative", "lo-fi indie", "indie", "indie folk", "indietronica", 
                       "indie pop", "indie rock", "rock and indie"],
"electronic/dance": ["dance and electronica", "electro house", "electronic", "electropop", 
                      "progressive house", "hip house", "house", "eurodance", "dancehall", "dance", "trap", "funk carioca"],
"soul": ["psychedelic soul", "deep soul", "neo-soul", "neo soul", "southern soul", 
          "smooth soul", "blue-eyed soul", "soul and reggae", "soul", "funk"],
"classical/soundtrack": ["classical", "orchestral", "film soundtrack", "composer"],
"pop": ["country-pop", "latin pop", "classical pop", "pop-metal", "orchestral pop", 
         "instrumental pop", "indie pop", "sophisti-pop", "pop punk", "pop reggae", "noah cyrus"
         "britpop", "traditional pop", "power pop", "sunshine pop", "baroque pop", "alternative r&b",
         "synthpop", "art pop", "teen pop", "psychedelic pop", "folk pop", "country pop", 
         "pop rap", "pop soul", "pop and chart", "dance-pop", "pop", "top 40", "american idol", 'breakeven', 'm2m'],
"hip-hop/rnb": ["conscious hip hop", "east coast hip hop", "hardcore hip hop", 
                 "west coast hip hop", "hiphop", "southern hip hop", "hip-hop", 
                 "hip hop", "hip hop rnb and dance hall", "contemporary r b", 
                 "gangsta rap", "rapper", "rap", "rhythm and blues", "contemporary rnb", 
                 "contemporary r&b", "rnb", "rhythm & blues","r&b", "blues", "mumble rapper", 'michael jackson'],
"disco": ["disco"],
"swing":  ["swing"],
"folk": ["contemporary folk", "folk", "bachata", "latin", "romanian"],
"country": ["country rock", "country-pop", "country pop", "contemporary country", "country"],
"jazz": ["vocal jazz", "jazz", "jazz-rock"],
"religious": ["christian", "christmas music", "gospel"],
"blues": ["delta blues", "rock blues", "urban blues", "electric blues",
           "acoustic blues", "soul blues", "country blues", "jump blues", 
           "classic rock", "blues rock", "jazz and blues", "piano blues", 
           "british blues", "british rhythm & blues", "rhythm and blues", "blues", "blues rock", "rhythm & blues"],
"reggae": ["reggae fusion", "roots reggae", "reggaeton", "pop reggae", "reggae", "soul and reggae"]}

In [ ]:
all_subgenres=set([i for i in list(artist_genres_df.genres.values) for i in i])

In [ ]:
[i for i in artist_genres_df.loc[artist_genres_df['artist'] == 'Lil Wayne'].genres for i in i]

In [ ]:
df = pd.read_csv("../data/billboard.csv", index_col = 0)
df = df[df.date >= "2000-01-01"]
df

In [ ]:
def determine_genre(subgenres) :
    for subgenre in subgenres :
        for genre, subgrs in aggregate_genres.items():
            if subgenre in subgrs:
                return genre
    return np.NaN

In [37]:
def determine_genre(subgenres) :
    for subgenre in subgenres :
        for genre, subgrs in aggregate_genres.items():
            if subgenre in subgrs:
                return genre
    return np.NaN

```
def from_artist_to_subgenres(artist):
    subgenres = artist_genres_df.loc[artist_genres_df['artist'] == artist].genres 
    if subgenres.empty:
        return np.NaN
    else :
        return [i for i in subgenres for i in i]
```

In [ ]:
df['subgenres'] = df['artist'].map(from_artist_to_subgenres)

In [ ]:
df

<h4>Weight formula applied to determine the subgenre popularity </h4>



Popularity of each subgenre is initialized at 0.  
$s$ is a linear score given accordingly to the rank of the subgenre occurrence.


Then, for each occurrence of the genre in the charts we have :  
$ \widehat{popularity_{subgenre}} = popularity_{subgenre} +s$
  
 where $s = 1 / rank $

In [ ]:
subgenres_popularity = dict({e:0 for e in [i for i in list(aggregate_genres.values()) for i in i]})

In [ ]:
for index, row in df.loc[pd.notna(df['subgenres'])].iterrows():
    for subg in row['subgenres']:
        try : 
            subgenres_popularity[subg] += 1/row['rank']
        except KeyError:
            continue
        
    
    # print(index, row['rank'], row['subgenres'])

In [ ]:
# subgenres_popularity = {key:value for (key,value) in subgenres_popularity.items() if value>0}

In [ ]:
f = open("hey.txt", "w")

f.write("series: [")

for genre in aggregate_genres.keys():
    f.write("\t{\n")
    f.write("\tname : "+"'"+genre+"'"+",\n")
    f.write("\tdata: [")
    
    for subgenre in aggregate_genres[genre] : 
        f.write("{\n")
        f.write("\t\tname: "+"'"+subgenre+"',\n")
        f.write("\t\tvalue: "+str(subgenres_popularity[subgenre])+"\n")
        f.write("\t},")
    
    f.write('\t]\n')   #  closing data 
    f.write("\t},\n")

f.write("]")

f.close()

In [ ]:
separate_artists(artist_genres_df)